In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_24179/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/20 00:36:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/20 00:36:39 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for Transactions table
transactions_data = {
    'transaction_id': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'customer_id': [101, 101, 101, 102, 102, 102, 105, 105, 105],
    'transaction_date': ['2023-05-01', '2023-05-02', '2023-05-03', '2023-05-01', '2023-05-03',
                         '2023-05-04', '2023-05-01', '2023-05-02', '2023-05-03'],
    'amount': [100, 150, 200, 50, 100, 200, 100, 150, 200]
}

# Create DataFrame for Transactions
transactions_df = pd.DataFrame(transactions_data)

# Convert transaction_date to datetime type
transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'])

# Display the DataFrame
print("Transactions table:")
print(transactions_df)



df_person = spark.createDataFrame(transactions_df)
df_person.createOrReplaceTempView("Transactions")



Transactions table:
   transaction_id  customer_id transaction_date  amount
0               1          101       2023-05-01     100
1               2          101       2023-05-02     150
2               3          101       2023-05-03     200
3               4          102       2023-05-01      50
4               5          102       2023-05-03     100
5               6          102       2023-05-04     200
6               7          105       2023-05-01     100
7               8          105       2023-05-02     150
8               9          105       2023-05-03     200


In [14]:
query = """
    select
        customer_id
    from (
        select
            customer_id,
            rank() over(order by number_in_group desc) as rk
        from (
            select
                customer_id,
                count(*) as number_in_group
            from (
                select
                    *,
                    sum(
                        if(is_consecutive_day, 0, 1)
                    ) over(partition by customer_id order by transaction_date asc) as group_id
                from (
                    select
                        *,
                        (date_diff(transaction_date, previous_transaction_date) = 1) and (amount > previous_amount) as is_consecutive_day
                    from (
                        select
                            *,
                            lag(transaction_date, 1, transaction_date) over(partition by customer_id order by transaction_date asc) as previous_transaction_date,
                            lag(amount, 1, amount) over(partition by customer_id order by transaction_date asc) as previous_amount
                        from Transactions 
                    )
                )
            ) 
            group by customer_id, group_id
        )
    ) where rk = 1
"""

In [15]:
spark.sql(query).show()

24/03/20 00:49:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/20 00:49:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/20 00:49:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/20 00:49:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/20 00:49:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


24/03/20 00:49:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/20 00:49:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+
|customer_id|
+-----------+
|        101|
|        105|
+-----------+

